In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
from scipy.io import wavfile
from pydub import AudioSegment

# =======================================
# Kelas STA-LTA
# =======================================
class STALTA:
    def _init_(self, data, fs, sta_window=0.1, lta_window=1.0):
        self.data = data
        self.fs = fs
        self.sta_window = sta_window
        self.lta_window = lta_window
        self.sta_n = int(sta_window * fs)
        self.lta_n = int(lta_window * fs)

    def compute(self):
        sta = np.convolve(self.data ** 2, np.ones(self.sta_n), 'same') / self.sta_n
        lta = np.convolve(self.data ** 2, np.ones(self.lta_n), 'same') / self.lta_n
        lta[lta == 0] = 1e-12
        ratio = sta / lta
        return ratio

# =======================================
# GUI
# =======================================
class STA_LTA_App:
    def _init_(self, root):
        self.root = root
        self.root.title("STA/LTA Viewer + Arsiran Batas")
        self.root.geometry("400x220")

        self.label_sta = tk.Label(root, text="STA window (detik):")
        self.label_sta.pack()
        self.entry_sta = tk.Entry(root)
        self.entry_sta.insert(0, "0.1")
        self.entry_sta.pack()

        self.label_lta = tk.Label(root, text="LTA window (detik):")
        self.label_lta.pack()
        self.entry_lta = tk.Entry(root)
        self.entry_lta.insert(0, "1.0")
        self.entry_lta.pack()

        self.btn_load = tk.Button(root, text="Pilih File Audio", command=self.load_file)
        self.btn_load.pack(pady=10)

        self.label_info = tk.Label(root, text="Belum ada file")
        self.label_info.pack()

    def load_file(self):
        path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.wav *.mp3")])
        if not path:
            return

        # Baca file
        if path.endswith(".wav"):
            fs, data = wavfile.read(path)
            if data.ndim > 1:
                data = data.mean(axis=1)
        else:
            audio = AudioSegment.from_file(path)
            data = np.array(audio.get_array_of_samples()).astype(float)
            fs = audio.frame_rate
            if audio.channels > 1:
                data = data.reshape((-1, audio.channels)).mean(axis=1)

        data = data / np.max(np.abs(data))
        t = np.arange(len(data)) / fs

        # Ambil parameter STA dan LTA dari GUI
        try:
            sta_win = float(self.entry_sta.get())
            lta_win = float(self.entry_lta.get())
        except ValueError:
            sta_win, lta_win = 0.1, 1.0

        # Hitung STA/LTA
        stalta = STALTA(data, fs, sta_win, lta_win)
        ratio = stalta.compute()

        # Plot dengan arsiran
        plt.figure(figsize=(10, 6))
        plt.subplot(2, 1, 1)
        plt.plot(t, data, color='blue')
        plt.title("Sinyal Audio")
        plt.xlabel("Waktu (detik)")
        plt.ylabel("Amplitudo")

        plt.subplot(2, 1, 2)
        plt.plot(t, ratio, color='orange', label='STA/LTA')
        plt.title("Rasio STA/LTA")
        plt.xlabel("Waktu (detik)")
        plt.ylabel("Rasio")

        # Tambahkan arsiran batas
        plt.axvspan(0, sta_win, color='red', alpha=0.2, label='STA window')
        plt.axvspan(0, lta_win, color='blue', alpha=0.1, label='LTA window')
        plt.legend()

        plt.tight_layout()
        plt.show()

        self.label_info.config(text=f"File: {path.split('/')[-1]}")

# =======================================
# MAIN
# =======================================
if _name_ == "_main_":
    root = tk.Tk()
    app = STA_LTA_App(root)
    root.mainloop()